In [1]:
#
# Copyright (c) 2024, NVIDIA CORPORATION.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#

<img src="http://developer.download.nvidia.com/notebooks/dlsw-notebooks/tensorrt_torchtrt_efficientnet/nvidia_logo.png" width="90px">

# Distributed Hyperparameter Tuning: Optuna + JoblibSpark


This demo demonstrates distributed hyperparameter tuning for XGBoost using the [JoblibSpark backend](https://github.com/joblib/joblib-spark), building on this [example from Databricks](https://docs.databricks.com/en/machine-learning/automl-hyperparam-tuning/optuna.html).  
We implement best practices to precompute data and maximize computations on the GPU.  



Reference: https://forecastegy.com/posts/xgboost-hyperparameter-tuning-with-optuna/

#### Note:
Before running, please make sure you've followed the relevant [setup instructions](../README.md) for your environment (standalone or databricks).


In [2]:
from typing import List
import json
import os
import requests
import joblib
from joblibspark import register_spark
import optuna
from optuna.samplers import TPESampler
import xgboost as xgb
from pyspark.sql import SparkSession
from pyspark import TaskContext, SparkConf

### Download the dataset

We'll use the [red wine quality dataset](https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv) to regress wine quality based on features such as acidity, sugar content, etc.  

**Note**: This example uses a small dataset for demonstration purposes. The performance advantages of distributed training are best realized with large datasets and computational workloads.

In [3]:
cwd = os.getcwd()
os.mkdir(os.path.join(cwd, "data")) if not os.path.exists(os.path.join(cwd, "data")) else None
filepath = os.path.join(cwd, "data", "winequality-red.csv")

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"

response = requests.get(url)
if response.status_code == 200:
    with open(filepath, "wb") as f:
        f.write(response.content)
    print(f"File downloaded and saved to {filepath}")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")

## Part 1. Running Optuna locally

In [5]:
import cudf
from cuml.metrics.regression import mean_squared_error
from cuml.model_selection import train_test_split

### Prepare data

In [6]:
data = cudf.read_csv(filepath, delimiter=";")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Prepare the train/validation sets. Precompute the Quantile DMatrix, which is used by histogram-based tree methods to save memory.

In [7]:
X = data.iloc[:, :-1].values
y = data["quality"].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
Xy_train_qdm = xgb.QuantileDMatrix(X_train, y_train)  # Precompute Quantile DMatrix to avoid repeated quantization every trial.

### Objective function

We define the objective and a hyperparameter search space to optimize via the `trial.suggest_` methods.  

In each trial, new hyperparameters will be suggested based on previous results. See [optuna.trial.Trial](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.trial.Trial.html) API for a full list of functions.

In [8]:
def objective(trial):
    params = {
        "objective": "reg:squarederror",
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        "max_bins": 64,
        "tree_method": "gpu_hist",
        "device": "cuda",
    }

    booster = xgb.train(params, dtrain=Xy_train_qdm, num_boost_round=1000)
    predictions = booster.inplace_predict(X_val)
    rmse = mean_squared_error(y_val, predictions, squared=False).get()
    
    return rmse   

Create the study and optimize. By default, the study results will be stored in memory.

In [9]:
study = optuna.create_study(study_name="optuna-xgboost-local", sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=10)

[I 2024-12-06 19:47:02,101] A new study created in memory with name: optuna-xgboost-local


[I 2024-12-06 19:47:03,955] Trial 0 finished with value: 0.5448661551754318 and parameters: {'learning_rate': 0.005611516415334507, 'max_depth': 10, 'subsample': 0.7453942447208348, 'colsample_bytree': 0.6187255599871848, 'min_child_weight': 4}. Best is trial 0 with value: 0.5448661551754318.
[I 2024-12-06 19:47:04,191] Trial 1 finished with value: 0.6762149201768457 and parameters: {'learning_rate': 0.002051110418843397, 'max_depth': 1, 'subsample': 0.8728673384861885, 'colsample_bytree': 0.6210592611560484, 'min_child_weight': 15}. Best is trial 0 with value: 0.5448661551754318.
[I 2024-12-06 19:47:05,920] Trial 2 finished with value: 0.6871436852105118 and parameters: {'learning_rate': 0.0010994335574766201, 'max_depth': 10, 'subsample': 0.8408205087604007, 'colsample_bytree': 0.25172215514436236, 'min_child_weight': 4}. Best is trial 0 with value: 0.5448661551754318.
[I 2024-12-06 19:47:06,505] Trial 3 finished with value: 0.6096809835807359 and parameters: {'learning_rate': 0.0023

In [10]:
trial = study.best_trial
print("Best RMSE: ", trial.value)
print("Best hyperparameters: ", trial.params)

Best RMSE:  0.5448661551754318
Best hyperparameters:  {'learning_rate': 0.005611516415334507, 'max_depth': 10, 'subsample': 0.7453942447208348, 'colsample_bytree': 0.6187255599871848, 'min_child_weight': 4}


## Part 2. Distributed Optuna on Spark 

### Optuna Task

Define the task UDF to distribute across the Spark cluster. In each task, the worker will:
1. Read the dataset from the specified filepath.
2. Load the study from the MySQL storage backend.
3. Optimize over the objective for the assigned number of trials, sending results back to the database after each iteration.

In [11]:
def task(num_trials: int, driver_ip: str, study_name: str, seed: int, filepath: str):
    import cudf
    from cuml.metrics.regression import mean_squared_error
    from cuml.model_selection import train_test_split

    def get_gpu_id(task_context: TaskContext) -> int:
        if task_context is None:
            raise RuntimeError("_get_gpu_id should not be invoked from driver side.")
        resources = task_context.resources()

        if "gpu" not in resources:
            raise RuntimeError(
                "Couldn't get the gpu id, Please check the GPU resource configuration"
            )
        
        return int(resources["gpu"].addresses[0].strip())  # Return the first GPU ID for multi-GPU setups.

    tc = TaskContext.get()
    gpu_id = get_gpu_id(tc)

    if filepath.startswith("/dbfs/"):
        # Check to ensure GPU direct storage is disabled for cuDF on databricks.
        libcudf_policy = os.environ.get('LIBCUDF_CUFILE_POLICY')
        if libcudf_policy != 'OFF':
            raise RuntimeError("Set LIBCUDF_CUFILE_POLICY=OFF to read from DBFS with cuDF.")
    
    data = cudf.read_csv(filepath, delimiter=";")
    X = data.iloc[:, :-1].values
    y = data["quality"].values
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)
    Xy_train_qdm = xgb.QuantileDMatrix(X_train, y_train)  # Precompute Quantile DMatrix to avoid repeated quantization every trial.

    def objective(trial):
        params = {
            "objective": "reg:squarederror",
            "verbosity": 0,
            "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
            "max_depth": trial.suggest_int("max_depth", 1, 10),
            "subsample": trial.suggest_float("subsample", 0.05, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
            "max_bins": trial.suggest_int("max_bins", 32, 256),
            "tree_method": "gpu_hist",
            "device": f"cuda:{gpu_id}",
            "seed": seed,
        }

        if "max_bins" in params:
            # If tuning the max_bins param, we must recompute the QDM every trial.
            if "num_estimators" not in params:
                params["num_estimators"] = 1000

            model = xgb.XGBRegressor(**params)
            model.fit(X_train, y_train)
            booster = model.get_booster()
        else:
            # Train the model with xgb.train() API using the precomputed QDM.
            booster = xgb.train(params, dtrain=Xy_train_qdm, num_boost_round=1000)
            
        # Perform in-place predictions on GPU using the booster.
        predictions = booster.inplace_predict(X_val)
        rmse = mean_squared_error(y_val, predictions, squared=False).get()
        
        return rmse

    study = optuna.load_study(
        study_name=study_name,
        storage=f"mysql://optuna_user:optuna_password@{driver_ip}/optuna",
        sampler=TPESampler(seed=seed),
    )

    print(f"Running {num_trials} trials on partition {tc.partitionId()}.")
    study.optimize(objective, n_trials=num_trials)

    return study.best_params, study.best_value

### PySpark

For standalone users, we need to create the Spark session with the Spark-Rapids plugin. For Databricks users, the Spark session will be preconfigured.

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

def initialize_spark():
    import socket
    hostname = socket.gethostname()
    conda_env = os.environ.get("CONDA_PREFIX")

    conf = SparkConf()
    conf.setMaster(f"spark://{hostname}:7077")  # Assuming master is on host and default port. 
    conf.set("spark.task.maxFailures", "1")
    conf.set("spark.task.resource.gpu.amount", "1")
    conf.set("spark.executor.resource.gpu.amount", "1")
    conf.set("spark.pyspark.python", f"{conda_env}/bin/python")
    conf.set("spark.pyspark.driver.python", f"{conda_env}/bin/python")
    
    spark = SparkSession.builder.appName("optuna-joblibspark-xgboost").config(conf=conf).getOrCreate()
    return spark

if 'spark' not in globals():
    spark = initialize_spark()

In [13]:
# This will register the Spark Session with the Joblib Spark backend.
register_spark()

## Setup and run the Optuna study

Get the driver IP for the MySQL database.  
- For standalone users, make sure you've followed the [database setup instructions](../README.md#setup-database-for-optuna). The database should be on 'localhost'. 
- For databricks users, the database should already be setup on the driver node by the init script.

In [14]:
# check if we're running on databricks
on_databricks = os.environ.get("DATABRICKS_RUNTIME_VERSION", False)

In [15]:
if on_databricks:
    driver_ip = spark.conf.get("spark.driver.host")
else:
    driver_ip = "localhost"

print(f"MySQL database is hosted on {driver_ip}")

MySQL database is hosted on localhost


Create a new study, referencing the MySQL database as the storage backend.

In [16]:
study_name = "optuna-xgboost-joblibspark"
seed = 42

try:
    # Delete the study if it already exists
    optuna.delete_study(
        study_name=study_name, 
        storage=f"mysql://optuna_user:optuna_password@{driver_ip}/optuna"
    )
except:
    pass

optuna.create_study(
    study_name=study_name,
    storage=f"mysql://optuna_user:optuna_password@{driver_ip}/optuna",
    sampler=TPESampler(seed=seed)
)

[I 2024-12-06 19:47:11,323] A new study created in RDB with name: optuna-xgboost-joblibspark


Define the number of tasks, number of trials, and trials per task. 

**NOTE**: for standalone users running on a single worker, the 4 tasks will all be assigned to the same worker and executed sequentially in this demonstration. This can easily be scaled up by adding more workers.

In [17]:
def partition_trials(total_trials: int, total_tasks: int) -> List[int]:
    base_size = total_trials // total_tasks
    extra = total_trials % total_tasks
    partitions = [base_size] * total_tasks
    for i in range(extra):
        partitions[i] += 1
    
    return partitions

In [18]:
num_tasks = 4
num_trials = 100
trials_per_task = partition_trials(num_trials, num_tasks)
print(f"Trials per task: {trials_per_task}")

Trials per task: [25, 25, 25, 25]


**For Databricks**: we must download the dataset to DBFS so that all workers can access it.

In [19]:
if on_databricks:
    dbutils.fs.mkdirs("/FileStore/optuna-data")
    filepath = "/dbfs/FileStore/optuna-data/winequality-red.csv"
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"

    response = requests.get(url)
    if response.status_code == 200:
        with open(filepath, "wb") as f:
            f.write(response.content)
        print(f"File downloaded and saved to {filepath}")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")

### Run the study

In [ ]:
with joblib.parallel_backend("spark", n_jobs=num_tasks):
    results = joblib.Parallel()(
        joblib.delayed(task)(num_trials, driver_ip, study_name, seed, filepath) for num_trials in trials_per_task
    )

In [21]:
best_params = min(results, key=lambda x: x[1])[0]
best_value = min(results, key=lambda x: x[1])[1]

print(f"Best parameters: {best_params}")
print(f"Best value: {best_value}")

Best parameters: {'learning_rate': 0.06337865756914637, 'max_depth': 10, 'subsample': 0.8315729907914456, 'colsample_bytree': 0.885700420269793, 'min_child_weight': 18, 'max_bins': 72}
Best value: 0.5529384630337998
